# Import libraries

In [1]:
%gui qt5
%matplotlib qt5

import time
import os
from datetime import datetime

import numpy as np
import matplotlib.pyplot as mplt

from siriuspy.devices import SOFB, Tune, RFGen
from apsuite.commisslib.meas_ac_orm import MeasACORM

from mathphys.functions import save, load

mplt.rcParams.update({
    'font.size': 12,'lines.linewidth': 2,
    'axes.grid': True, 'grid.alpha': 0.5,
    'grid.linestyle': '--', 'text.usetex': True*0})

# Create Objects

In [122]:
meas_orm = MeasACORM(isonline=True)
sofb = SOFB(SOFB.DEVICES.SI)
tune = Tune(Tune.DEVICES.SI)

Creating kick converters  -> ET: = 2.69s
Creating correctors       -> ET: = 11.20s
Creating General Devices  -> ET: = 0.01s
Creating Timing           -> ET: = 0.04s
Creating BPMs             -> ET: = 3.29s


# Configure Measurement

In [123]:
meas_orm.params.timeout_bpms = 60
meas_orm.params.timeout_correctors = 20
meas_orm.params.ref_respmat_name = 'ref_respmat'
meas_orm.params.meas_bpms_noise = True
meas_orm.params.meas_rf_line = True*1

# meas_orm.params.corrs_ch2meas = meas_orm.sofb_data.ch_names[-1:] + meas_orm.sofb_data.ch_names[:5]  # 'all'
# meas_orm.params.corrs_cv2meas = meas_orm.sofb_data.cv_names[-1:] + meas_orm.sofb_data.cv_names[:7]  # 'all'
meas_orm.params.corrs_ch2meas = 'all'
meas_orm.params.corrs_cv2meas = 'all'
meas_orm.params.corrs_nrruns_per_acq = 4
meas_orm.params.corrs_excit_time = 4.000
meas_orm.params.corrs_delay = 5e-3
meas_orm.params.corrs_norm_kicks = True
meas_orm.params.corrs_ch_kick = 5.000
meas_orm.params.corrs_cv_kick = 5.000
meas_orm.params.corrs_dorb1ch = 40.000
meas_orm.params.corrs_dorb1cv = 40.000

nrsecs = 1
primes = meas_orm.params.find_primes(2*8*nrsecs + 2, 3)
primes = np.array(sorted(set(primes) - {59, 61}))
cv_freqs = primes[:8*nrsecs]
primes = np.array(sorted(set(primes) - set(cv_freqs)))
ch_freqs = primes[:6*nrsecs]
meas_orm.params.corrs_ch_freqs = ch_freqs
meas_orm.params.corrs_cv_freqs = cv_freqs

meas_orm.params.rf_excit_time = 1.000
meas_orm.params.rf_mode = 'Step'
meas_orm.params.rf_step_kick = 5.000
meas_orm.params.rf_step_delay = 0.5
meas_orm.params.rf_phase_amp = 2.000

In [124]:
print(meas_orm.params)

timeout_bpms             =    60.000  [s]
timeout_correctors       =    20.000  [s]
ref_respmat_name         = ref_respmat  
meas_bpms_noise          = True       
meas_rf_line             = 1          
corrs_ch2meas            = all        
corrs_cv2meas            = all        
corrs_nrruns_per_acq     =         4  
corrs_excit_time         =     4.000  [s]
corrs_delay              =     0.005  [s]
corrs_norm_kicks         = True       
corrs_ch_kick            =     5.000  [urad] (only used if corrs_norm_kicks == False)
corrs_cv_kick            =     5.000  [urad] (only used if corrs_norm_kicks == False)
corrs_dorb1ch            =    40.000  [um] (only used if corrs_norm_kicks == True)
corrs_dorb1cv            =    40.000  [um] (only used if corrs_norm_kicks == True)
corrs_ch_freqs           = [29 31 37 41 43 47]  [Hz]
corrs_cv_freqs           = [ 3  5  7 11 13 17 19 23]  [Hz]
rf_excit_time            =     1.000  [s]
rf_mode                  = Step       RFModes(Step=0, Phase=1)
rf

# Meas AC ORM

In [125]:
meas_orm.start()

Measuring BPMs Noise:
    Configuring BPMs...

Done! ET: 1.62s
    Sending Trigger signal...Done! ET: 0.48s
    Waiting BPMs to update...

In [127]:
meas_orm.ismeasuring

True

Done! ET: 30.20s
    Changing Correctors to SlowRef...Done! ET: 0.52s
  Elapsed Time: 34.40s
Finished!!  ET: 2.85min
All measurements finished!!


In [12]:
meas_orm.stop()

In [128]:
meas_orm.check_measurement_finished_ok()

True

In [129]:
meas_orm.process_data()

Processing Magnets Data:
  Acquisition 01/20 ET: 0.476126s
  Acquisition 02/20 ET: 0.163081s
  Acquisition 03/20 ET: 0.072383s
  Acquisition 04/20 ET: 0.062563s
  Acquisition 05/20 ET: 0.062864s
  Acquisition 06/20 ET: 0.062800s
  Acquisition 07/20 ET: 0.062985s
  Acquisition 08/20 ET: 0.063000s
  Acquisition 09/20 ET: 0.061603s
  Acquisition 10/20 ET: 0.061980s
  Acquisition 11/20 ET: 0.064113s
  Acquisition 12/20 ET: 0.063280s
  Acquisition 13/20 ET: 0.062886s
  Acquisition 14/20 ET: 0.067632s
  Acquisition 15/20 ET: 0.063431s
  Acquisition 16/20 ET: 0.063236s
  Acquisition 17/20 ET: 0.063589s
  Acquisition 18/20 ET: 0.062005s
  Acquisition 19/20 ET: 0.064503s
  Acquisition 20/20 ET: 0.062744s
Done processing Magnets Data!
Processing RF Step...Done! ET: 0.027526s
Processing BPMs Noise...Done! ET: 0.180273s


In [133]:
meas_orm.check_measurement_quality()

True

In [131]:
ac_respmat = meas_orm.build_respmat()

In [132]:
meas_orm.save_loco_input_data('meas_at_200mA_21_06_2025', matrix=ac_respmat)

In [134]:
ref_respmat = meas_orm.get_ref_respmat()

# Analysis

In [136]:
mplt.figure(figsize=(10, 4))
idx = 0
mplt.plot(ref_respmat[:, idx], label='Ref', ls='-')
mplt.plot(ac_respmat[:, idx], label='AC', ls='--')
mplt.legend()
mplt.show()

### Scale Factors

In [36]:
fig, ax = meas_orm.plot_scale_conversion_factors()

### Correlation Residues

In [38]:
mat_ac = ac_respmat
mat_dc = ref_respmat

In [44]:
fig, axs, corr = meas_orm.plot_comparison_correlations(mat_ac=mat_ac, mat_dc=mat_dc)
corr_h = corr[:, :120]
corr_v = corr[:, 120:280]
corr_rf = corr[:, -1]

Done! ET: 3.91s
    Elapsed Time: 6.93s
Measuring Magnets:
  Acquisition 01/05
    Configuring BPMs and timing...Done! ET: 1.65s
    Configuring Correctors...Done! ET: 0.51s
    Changing Correctors to Cycle...Done! ET: 0.33s
    Sending Timing signal...Done! ET: 0.25s
    Waiting BPMs to update...Done! ET: 27.73s
    Changing Correctors to SlowRef...Done! ET: 0.35s
  Elapsed Time: 31.21s
  Acquisition 02/05
    Configuring BPMs and timing...Done! ET: 1.57s
    Configuring Correctors...Done! ET: 0.49s
    Changing Correctors to Cycle...Done! ET: 0.43s
    Sending Timing signal...Done! ET: 0.80s
    Waiting BPMs to update...

In [29]:
idcsh = corr_h[1].argsort()
idcsv = corr_v[0].argsort() + 120

In [30]:
fig, axs = meas_orm.plot_comparison_single_corrector(idcsh[-1], mat_ac=mat_ac, mat_dc=mat_dc)
fig, axs = meas_orm.plot_comparison_single_corrector(idcsv[-1], mat_ac=mat_ac, mat_dc=mat_dc)

### Analyse Fitting

In [5]:
print('Done with Initiallization!')

Done with Initiallization!


In [25]:
fig, axs, *orbs = meas_orm.plot_orbit_residue_after_fitting(bpm_idx=0, excit_idx=0, time_domain=True)

In [26]:
fig, axs, *orbs = meas_orm.plot_orbit_residue_after_fitting(bpm_idx=0, excit_idx=0, time_domain=False)

### Analyse idx_ini

In [17]:
idx = -1
fig, axs = meas_orm.plot_phases_vs_amplitudes(title='', corrsidx2highlight=[idcsh[-1], idcsv[-1]])

In [18]:
fig, axs = meas_orm.plot_phases_histogram(title='')

In [51]:
meas_orm.ismeasuring


False

In [47]:
meas_orm.verbose = False